In [47]:
import gensim.downloader
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import spacy
from gensim.models import KeyedVectors
from keras.layers import Dense, Flatten, LSTM, GRU, Bidirectional
from keras.models import Sequential
from keras.preprocessing import sequence, text
from sklearn.metrics import precision_score, recall_score, f1_score, cohen_kappa_score, classification_report
from sklearn.model_selection import train_test_split
from spacy.tokenizer import Tokenizer

In [48]:
data = pd.read_csv('data\preprocessed.csv').dropna()

In [49]:
nlp = spacy.load("en_core_web_sm")
#glove = gensim.downloader.load('glove-twitter-200')
#glove.save("glove.model")
glove = KeyedVectors.load("glove.model")

In [50]:
def get_vector(sent):
    for word in sent.lower().split(' '):
        if word.isalpha() == True:
            try:
                return glove[word].tolist()
            except KeyError:
                return glove['unk'].tolist()
        else:
            return glove['unk'].tolist()

In [51]:
def pad_trunc(data, maxlen):
    new_data = []
    zero_vector = []
    for _ in range(len(data[0][0])):
        zero_vector.append(0.0)
    for sample in data:
        if len(sample) > maxlen:
            temp = sample[:maxlen]
        elif len(sample) < maxlen:
            temp = sample
            additional_elems = maxlen - len(sample)
            for _ in range(additional_elems):
                temp.append(zero_vector)
        else:
            temp = sample
        new_data.append(temp)
        
    return new_data

In [52]:
data['vectors'] =  data['Reviews_cleaned'].apply(lambda x: [get_vector(x)])

In [53]:
X = data['vectors']
y = data["Category"]
X = np.array(pad_trunc(X, 10))

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [55]:
X_train.shape, X_test.shape, X.shape

((32560, 10, 200), (8140, 10, 200), (40700, 10, 200))

In [56]:
model = Sequential()
model.add(Bidirectional(LSTM(32, return_sequences=True, input_shape=(10,200))))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [57]:
history = model.fit(X_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/2
204/204 [==============================] - 20s 58ms/step - loss: 0.4589 - accuracy: 0.8133 - val_loss: 0.4329 - val_accuracy: 0.8308
Epoch 2/2
204/204 [==============================] - 10s 49ms/step - loss: 0.4434 - accuracy: 0.8201 - val_loss: 0.4299 - val_accuracy: 0.8308


In [58]:
y_pred = np.round(model.predict(X_test))
cohen_kappa_score(y_test, y_pred)

0.5154885092632424

In [59]:
f1_score(y_test, y_pred)

0.8901271385967666

In [60]:
model = Sequential()
model.add(Bidirectional(GRU(32, return_sequences=True, input_shape=(10,200))))
model.add(GRU(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2)

Epoch 1/2
204/204 [==============================] - 16s 45ms/step - loss: 0.4630 - accuracy: 0.8108 - val_loss: 0.4356 - val_accuracy: 0.8308
Epoch 2/2
204/204 [==============================] - 8s 39ms/step - loss: 0.4458 - accuracy: 0.8201 - val_loss: 0.4299 - val_accuracy: 0.8308


In [61]:
y_pred = np.round(model.predict(X_test))
cohen_kappa_score(y_test, y_pred)

0.5154885092632424

In [62]:
f1_score(y_test, y_pred)

0.8901271385967666